##### So this program will detect if an email is spam (1) or not (0)
###### import libraries

In [1]:
import os
import numpy as np
import pandas as pd
import nltk
from  nltk.corpus import stopwords
import string

In [2]:
#setting the working directory
os.chdir(r"C:\Users\thars\Downloads")

In [3]:
df = pd.read_csv(r"C:\Users\thars\Downloads\emails.csv")
print(df.head())

                                                text  spam
0  Subject: naturally irresistible your corporate...     1
1  Subject: the stock trading gunslinger  fanny i...     1
2  Subject: unbelievable new homes made easy  im ...     1
3  Subject: 4 color printing special  request add...     1
4  Subject: do not have money , get software cds ...     1


In [4]:
#let's explore the data and get number of rows and columns
df.shape

(5728, 2)

In [5]:
#let's get the columns name
df.columns

Index(['text', 'spam'], dtype='object')

In [10]:
#checking the duplicate rows
df.loc[df.duplicated()]

,text,spam
2155,Subject: research allocations to egm hi becky...,0
2260,Subject: departure of grant masson the resear...,0
2412,"Subject: re : schedule and more . . jinbaek ,...",0
2473,"Subject: day off tuesday stinson , i would l...",0
2763,"Subject: re : your mail zhendong , dr . kami...",0
3123,"Subject: re : grades pam , the students rese...",0
3152,Subject: tiger evals - attachment tiger hosts...,0
3248,"Subject: re : i am zhendong zhendong , thank...",0
3249,Subject: hello from enron dear dr . mcmullen ...,0
3387,"Subject: term paper dr . kaminski , attached...",0


In [12]:
#counting the duplicate rows
df.duplicated().sum()

33

In [13]:
#removig the duplicates rows
df.drop_duplicates(inplace=True)

In [14]:
#checking again duplicates if there any
df.duplicated().sum()

0

In [15]:
#checking shape 
df.shape

(5695, 2)

##### Now download the stop words

In [16]:
#stop words in the natural language processing are useless words (data)
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thars\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
#Now Create a function to clean the text and return the tokens. The cleaning of the text can be done by first removing punctuation and 
# #then removing the useless words also known as stop words.
def process(text):
    nonpunc = [char  for char in text  if char not in string.punctuation] 
    nonpunc = "".join(nonpunc)
    clean = [word for word in nonpunc.split() if word.lower() not in stopwords.words("english")]
    return clean

df["text"].head().apply(process)


0    [Subject, naturally, irresistible, corporate, ...
1    [Subject, stock, trading, gunslinger, fanny, m...
2    [Subject, unbelievable, new, homes, made, easy...
3    [Subject, 4, color, printing, special, request...
4    [Subject, money, get, software, cds, software,...
Name: text, dtype: object

##### Now converts the text into matrix of token counts

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
message = CountVectorizer(analyzer=process).fit_transform(df["text"])

In [29]:
#split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(message, df["spam"], test_size=0.2, random_state=0)

#to see the shape of the data
print(message.shape)

(5695, 37229)


In [37]:
#create and train the naive bayes classifier
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(xtrain, ytrain)

#to see the classifier prediction on the actual data set
print(classifier.predict(xtrain))
print(ytrain.values)


[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [38]:
#Now let’s see how well our model performed by evaluating the Naive Bayes classifier and the report, confusion matrix & accuracy score.
#Evaluating the model on the training dataset
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(xtrain)
print(classification_report(ytrain, pred))
print()
print("Confusion Matrix \n:", confusion_matrix(ytrain, pred))
print("Accuracy Score \n:", accuracy_score(ytrain, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3457
           1       0.99      1.00      0.99      1099

    accuracy                           1.00      4556
   macro avg       0.99      1.00      1.00      4556
weighted avg       1.00      1.00      1.00      4556


Confusion Matrix 
: [[3445   12]
 [   1 1098]]
Accuracy Score 
: 0.9971466198419666


In [39]:
#Let’s test the model on the test data set (xtest &  ytest) by printing the predicted value, and the actual value to 
# see if the model can accurately classify the email text.
print(classifier.predict(xtest))
print(ytrain.values)

[1 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [40]:
## Evaluating the model on the training data set
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(xtest)
print(classification_report(ytest, pred))
print()
print("Confusion Matrix \n:", confusion_matrix(ytest, pred))
print("Accuracy Score \n:", accuracy_score(ytest, pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       870
           1       0.97      1.00      0.98       269

    accuracy                           0.99      1139
   macro avg       0.98      0.99      0.99      1139
weighted avg       0.99      0.99      0.99      1139


Confusion Matrix 
: [[862   8]
 [  1 268]]
Accuracy Score 
: 0.9920983318700615
